# Topic Clustering with LDA and Gaussian LDA
*Zijian Chen zc2386*  
*Yiran Wang yw3201*  
*Columbia University*
## 1. Introduction
**GOAL**  
The goal in modeling a large corpus of discrete documents is to find the latent clustering pattern of those documents as we assume that each document has a probability distribution over certain number of topics, and the words in each documents are drawn from document’s topic distribution. It is hard because we are not sure about the word distribution whether they are discrete or continuous, nor do we have any information about the number of topics in many cases.  
**DATA**  
To solve topic clustering problem we access NIPS corpus data NIPS12.

In [1]:
import edward as ed

## 2. LDA Model 

### 2.1 Modeling
LDA assumes the words in every documents are exchangeable, and each of which is a multinomial distribution over vocabularies of size $V$. The likelihood of a corpus $\mathbf{D}$ contains $D$ documents is given as following:  
\begin{align}
	p(\mathbf{D}; \alpha, \beta)=p(\theta, \beta, z, w)&=\prod_{k=1}^{K} p(\beta_{k};\eta)\prod_{d=1}^{D} p(\theta_{d} ; \alpha)(\prod_{n=1}^{N}p(z_{dn} | \theta_{d})p(w_{dn} |z_{dn},\beta_{z_{dn}})\\
	&= \prod_{k=1}^{K} p(\beta_{k};\eta)\prod_{d=1}^{D}p(\theta_{d} ; \alpha)(\prod_{n=1}^{N}\theta_{z_{dn}}f(w_{dn};b_{z_{dn}}))
\end{align}

**LDA generative process: ** 
1. For each topic $k \in \{1, 2, ..., K\}$, draw $\beta_{k} \sim Dirichlet(\eta)$  
2. For each document $d \in \{1, 2, ...,D\}$:  
    a) Draw $\theta_{d} \sim Dirichlet (\alpha)$  
    b) For each word $w_{n}$, $n \in \{1, 2, ..., N_{d}\}$ in document $d$:  
    * Draw $z_{dn} \sim Categorical(\theta_{d})$.  
    * Generaste word $w_{dn}\sim p(w_{dn} | z_{dn}, \beta_{z_{dn}})$, a categorical probability conditioned on the topic $z_{dn}$.  


<img src="LDABlei.jpg" width="600"/>

### 2.2 Collapsed Gibbs Sampling Inference for LDA  

### 2.3 Criticism for LDA

## 3. Gaussian LDA Model 

### 3.1 Modeling

### 3.2 Black Box Variational Inference Guassian LDA  